<a href="https://colab.research.google.com/github/spps-supalerk/ChatBot/blob/main/ChatBotHoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pythainlp
!pip install deepcut
!pip install fuzzywuzzy

In [2]:
class StockInfoSearch():
  def __init__(self, all_symbol:list, question_cat, question, general_question_cat, general_question, general_answer_cat, general_answer, answer_df, Tfidfvec_q, Tfidfvec_a):
    # Import necessary library
    import re
    from pythainlp.corpus import thai_stopwords
    from fuzzywuzzy import fuzz 
    from fuzzywuzzy import process
    from pythainlp.tokenize import word_tokenize
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity

    # Set Class Attribute
    self.all_symbol = all_symbol
    self.question_cat = question_cat
    self.question = question
    self.answer_df = answer_df
    self.th_stopwords = thai_stopwords()
    self.general_question_cat = general_question_cat
    self.general_question = general_question
    self.general_answer_cat = general_answer_cat
    self.general_answer = general_answer
    self.Tfidfvec_q = Tfidfvec_q
    self.Tfidfvec_a = Tfidfvec_a
    self.match_index_nostk = None
    self.query = None
    self.final_stock_name = None
    self.cleaned_question = None
    self.final_question = None
    self.match_index_noans = None
    self.final_answer = None

  def get_query(self, query):
    # deal with upper case
    self.query = query.upper()

  def get_stock_name(self):
    # find all possible stock name
    names_lst = re.findall('[A-Z1-9-]+', self.query)
    stock_names = [name.strip() for name in names_lst if name in self.all_symbol]

    # check no. stock & check PE
    len_stock_names = len(stock_names)
    if len_stock_names == 1:
      if 'PE' in stock_names:
        while True:
          stock_name_pe = input('น้องมีเงิน : ต้องการดูหุ้นที่ชื่อว่า PE กด 1 \nต้องการดู Price to Earning (P/E) กด 2\nหยุดการทำงาน กด enter\n')
          if stock_name_pe == '1':
            self.final_stock_name = stock_names
            break            

          elif stock_name_pe == '2':
            stock_names.remove('PE')
            self.final_stock_name = 'No stock value'
            break

          elif stock_name_pe == '':
            self.final_stock_name = ''
            break

          else:
            print('น้องมีเงิน : กรุณากรอกรหัสให้ถูกต้อง\n')        
      else:
        self.final_stock_name = stock_names

    elif len_stock_names >= 2:
      if 'PE' in stock_names:
        while True:
          stock_name_pe = input('น้องมีเงิน : ต้องการดูหุ้นที่ชื่อว่า PE กด 1 \nต้องการดู Price to Earning (P/E) กด 2\nหยุดการทำงาน กด enter\n')
          if stock_name_pe == '1':
            self.final_stock_name = stock_names
            break            

          elif stock_name_pe == '2':
            stock_names.remove('PE')
            self.final_stock_name = stock_names
            break

          elif stock_name_pe == '':
            self.final_stock_name = ''
            break

          else:
            print('น้องมีเงิน : กรุณากรอกรหัสให้ถูกต้อง\n')

    elif len_stock_names == 0:
      self.final_stock_name = 'No stock value'

  def get_question(self):
    # get question from query
    temp_query = self.query
    if self.final_stock_name != 'No stock value':
      if type(self.final_stock_name) == list:
        for name in self.final_stock_name:
          temp_query = temp_query.replace(name, '', 1).strip()
      else:
        temp_query = temp_query.replace(self.final_stock_name, '', 1).strip()
    else:
      pass

    self.cleaned_question = temp_query.lower()
    for stop_w in self.th_stopwords:
      self.cleaned_question.replace(stop_w,'')

  def get_mapping_question(self):
    # Generate question
    if self.final_stock_name == 'No stock value':
      self.general_question.append(self.cleaned_question)
      tfidf_q = Tfidfvec_q.transform(self.general_question)
      vals = cosine_similarity(tfidf_q[-1], tfidf_q)
      idx = vals.argsort()[0][-2]
      flat = vals.flatten()
      flat.sort()
      req_tfidf = flat[-2]
      if (req_tfidf>=0.3):
        self.general_answer.append(self.cleaned_question)
        tfidf_a = Tfidfvec_a.fit_transform(self.general_answer)
        vals = cosine_similarity(tfidf_a[-1], tfidf_a)
        idx = vals.argsort()[0][-2]
        flat = vals.flatten()
        flat.sort()
        req_tfidf = flat[-2]
        if (req_tfidf>=0.3):
          self.final_question = "No Answer"
        else:
          self.final_answer = self.general_answer[idx]
          self.final_question = self.general_answer_cat[idx]
      else:
        self.final_answer = self.general_answer[self.general_answer_cat.index(self.general_question_cat[idx])]
        self.final_question = self.general_question_cat[idx]
      self.general_question.remove(self.cleaned_question)
    else:
      match = process.extractOne(self.cleaned_question.lower(), self.question)[0]
      match_index = self.question.index(match)
      self.final_question = self.question_cat[match_index]
    
  def get_result(self):
    # output result
    if self.final_stock_name == 'No stock value':
      if self.final_question == "No Answer":
        print("น้องมีเงิน: ขอโทษที มีเงินไม่เข้าใจ")
      else:
        print("น้องมีเงิน: ไปหาคำตอบมาให้แล้วนะ")
        print(self.final_answer)
    else:
      temp_df = self.answer_df.copy()
      try:
        if self.final_stock_name == '':
          print('น้องมีเงิน: กรุณาตรวจสอบรหัสให้ถูกต้อง')
        else:
          if type(self.final_stock_name) == str:
            search_symbol = [self.final_stock_name]
          search_col = ['symbol'] + [self.final_question]
          final_result = temp_df[temp_df['symbol'].isin(self.final_stock_name)][search_col]
          print('น้องมีเงิน: ดูค่า "'+self.query+'" ได้ที่นี่เลย')
          display(final_result)
      except ValueError:
        print('น้องมีเงิน: กรุณาตรวจสอบรหัสให้ถูกต้อง')

  def greeting(self):
    # for greeting message
    greeting_input = ("hello", "hi", "hey", "สวัสดี", "ซาหวัดดี", "ดี", "หวัดดี", "ดีจ้า", "ไง", "ว่าไง", "เฮ้", "ฮัลโหล", "งาย", "โหล", "ทักครับ", "ทักค่ะ", "ภาหมี")
    greeting_output = ["หวัดดีจ้า", "สวัสดีจ้า", "สวัสดี", "ว่าไงคร้าบ", "ทักจ้ะ", "ดีใจจังมีคนคุยด้วย"]
    for word in self.query.split():
        if word.lower() in greeting_input:
            return random.choice(greeting_output)

In [3]:
# NLP Preprocessing
# from pythainlp.tokenize import sent_tokenize
from pythainlp.tokenize import word_tokenize
from pythainlp.util import normalize
from pythainlp.corpus import thai_stopwords
import re
import string
import io
import random
from fuzzywuzzy import fuzz 
from fuzzywuzzy import process

# Manipulate Data
import numpy as np
import pandas as pd

#Training TF-IDF
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Ignore Error
import warnings
warnings.filterwarnings('ignore')

# Load data from Google Drive
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

/usr/local/lib/python3.6/dist-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


## Get Stock Info

In [4]:
# Load SET factsheet from Google Drive
file_id = '1KR63nXmtTfKov7rfw_FMI8zQcHAtiKqQ'
output_name = 'SET_factsheet_20201109_165845.json'

downloaded = drive.CreateFile({'id':file_id})  
downloaded.GetContentFile(output_name)   

In [5]:
# Read SET facesheet to Pandas Dataframe
stock_info = pd.read_json('SET_factsheet_20201109_165845.json')

# Prepare SET facesheet Dataframe
stock_info[['52 week high','52 week low']] = stock_info['52 Week High/Low'].str.split("/",expand=True)
stock_info.drop(columns =['52 Week High/Low'], inplace = True)

stock_info.columns = ['symbol', 'name', 'market', 'industry', 'sector', 'price',
                      'p/e', 'p/bv', 'paid-up', 'market cap', 'ev',
                      'ebitda', 'ev/ebitda', 'url', 'scraped_time', '52 week high',
                      '52 week low']

## Get General Question

In [6]:
# Loat fundamental question and Q&A
file_id = '1bNJ-qhDE38_hxMBZRXViKFG01_IAtH4U'
output_name = 'question_mapping.xlsx'

downloaded2 = drive.CreateFile({'id':file_id})  
downloaded2.GetContentFile(output_name)

In [7]:
# Read fundamental question and Q&A to Pandas Dataframe
question_stock_mapping = pd.read_excel('question_mapping.xlsx',sheet_name='Stock Question')
answer_general_mapping = pd.read_excel('question_mapping.xlsx',sheet_name='General Question')
question_general_mapping = pd.read_excel('question_mapping.xlsx',sheet_name='General Question Mapping')

In [8]:
display(question_stock_mapping.head())
display(answer_general_mapping.head())
display(question_general_mapping.head())

,question_cat,question
0,name,คือบริษัท
1,name,ชื่อบริษัท
2,name,คือ
3,name,บริษัท
4,name,มีชื่อว่า


,question,answer
0,PE,Price to Earning : P/E คือ เป็นอัตราส่วนเปรียบ...
1,PBV,Price to Book Value : P/BV คือ เป็นอัตราส่วนเป...
2,Dividend,Dividend Yield เงินปันผล คือ เป็นการวัดอัตราผล...
3,Turnover,Turnover Ratio อัตราการหมุนเวียนการซื้อขาย ใช้...
4,Net Profit Margin,Net Profit Margin อัตราส่วนกำไรสุทธิ เป็นอัตรา...


,question_cat,question
0,PE,P/E คืออะไร
1,PBV,P/BV คืออะไร
2,Dividend,เงินปันผลคืออะไร
3,Dividend,Dividend คืออะไร
4,Turnover,Turnover คืออะไร


## TF-IDF For matching Q&A

In [9]:
# Fit TFIDF Vertorizer for Q&A - Question
Tfidfvec_q = TfidfVectorizer(encoding='utf-8')
Tfidfvec_q.fit(question_general_mapping['question'].values.tolist())

# Fit TFIDF Vertorizer for Q&A - Answer
Tfidfvec_a = TfidfVectorizer(encoding='utf-8')
Tfidfvec_a.fit(answer_general_mapping['question'].values.tolist())

TfidfVectorizer(analyzer='word', binary=False, decode_error='strict',
                dtype=<class 'numpy.float64'>, encoding='utf-8',
                input='content', lowercase=True, max_df=1.0, max_features=None,
                min_df=1, ngram_range=(1, 1), norm='l2', preprocessor=None,
                smooth_idf=True, stop_words=None, strip_accents=None,
                sublinear_tf=False, token_pattern='(?u)\\b\\w\\w+\\b',
                tokenizer=None, use_idf=True, vocabulary=None)

## Chatbot

In [11]:
sis = StockInfoSearch(all_symbol = stock_info['symbol'].unique(), question_cat =  question_stock_mapping['question_cat'].values.tolist(),
                      question  = question_stock_mapping['question'].values.tolist(),
                      general_question_cat = question_general_mapping['question_cat'].values.tolist(),
                      general_question = question_general_mapping['question'].values.tolist(), 
                      general_answer_cat = answer_general_mapping['question'].values.tolist(), 
                      general_answer = answer_general_mapping['answer'].values.tolist(),
                      answer_df = stock_info,
                      Tfidfvec_q = Tfidfvec_q, Tfidfvec_a = Tfidfvec_a)

flag=True
print("น้องมีเงิน: สวัสดี เราเป็น Chatbot ด้านหุ้น\n-> ถ้าอยากถามเราเกี่ยวกับรายละเอียดของหุ้น พิมพ์ชื่อหุ้น และคำถามที่ต้องการถามได้เลย เช่น 'PTT ราคาเท่าไร'\n-> หรือถ้ามีคำถามอื่นๆเกี่ยวกับเรื่องหุ้น ถามมาได้เลย เดี๋ยวมีเงินไปหามาให้\nแต่ถ้าไม่อยากคุยกับเราแล้ว พิมพ์ว่า 'ไปละ' มาได้เลย")
while(flag==True):
    user_response = input("User: ")
    if user_response!='ไปละ':
        if user_response in ['ขอบคุณ','แต้งกิ้ว','thanks','thank you']:
            flag=False
            print("น้องมีเงิน: ไม่เป็นไรจ้า")
        else:
            sis.get_query(user_response)
            if sis.greeting()!=None:
                print("น้องมีเงิน: "+sis.greeting())
            else:
                sis.get_stock_name()
                sis.get_question()
                sis.get_mapping_question()
                sis.get_result()
            print("น้องมีเงิน: ถ้ามีคำถามอื่นอีกบอกมาได้เลยน้าาา แต่ถ้าไม่อยากคุยกับเราแล้ว พิมพ์ว่า 'ไปละ' มาได้เลย")
    else:
        flag=False
        print("น้องมีเงิน: บะบายจ้า")

น้องมีเงิน: สวัสดี เราเป็น Chatbot ด้านหุ้น
-> ถ้าอยากถามเราเกี่ยวกับรายละเอียดของหุ้น พิมพ์ชื่อหุ้น และคำถามที่ต้องการถามได้เลย เช่น 'PTT ราคาเท่าไร'
-> หรือถ้ามีคำถามอื่นๆเกี่ยวกับเรื่องหุ้น ถามมาได้เลย เดี๋ยวมีเงินไปหามาให้
แต่ถ้าไม่อยากคุยกับเราแล้ว พิมพ์ว่า 'ไปละ' มาได้เลย
User: kbank price
น้องมีเงิน: ดูค่า "KBANK PRICE" ได้ที่นี่เลย


,symbol,price
310,KBANK,80.25


น้องมีเงิน: ถ้ามีคำถามอื่นอีกบอกมาได้เลยน้าาา แต่ถ้าไม่อยากคุยกับเราแล้ว พิมพ์ว่า 'ไปละ' มาได้เลย
User: aot pe
น้องมีเงิน : ต้องการดูหุ้นที่ชื่อว่า PE กด 1 
ต้องการดู Price to Earning (P/E) กด 2
หยุดการทำงาน กด enter
2
น้องมีเงิน: ดูค่า "AOT PE" ได้ที่นี่เลย


,symbol,p/e
44,AOT,60.47


น้องมีเงิน: ถ้ามีคำถามอื่นอีกบอกมาได้เลยน้าาา แต่ถ้าไม่อยากคุยกับเราแล้ว พิมพ์ว่า 'ไปละ' มาได้เลย
User: scc market cap
น้องมีเงิน: ดูค่า "SCC MARKET CAP" ได้ที่นี่เลย


...

น้องมีเงิน: ดูค่า "BDMS" ได้ที่นี่เลย


,symbol,name
84,BDMS,บริษัท กรุงเทพดุสิตเวชการ จำกัด(มหาชน)


น้องมีเงิน: ถ้ามีคำถามอื่นอีกบอกมาได้เลยน้าาา แต่ถ้าไม่อยากคุยกับเราแล้ว พิมพ์ว่า 'ไปละ' มาได้เลย
User: bdms ปันผล
น้องมีเงิน: ดูค่า "BDMS ปันผล" ได้ที่นี่เลย


,symbol,url
84,BDMS,https://www.set.or.th/set/factsheet.do?symbol=...


น้องมีเงิน: ถ้ามีคำถามอื่นอีกบอกมาได้เลยน้าาา แต่ถ้าไม่อยากคุยกับเราแล้ว พิมพ์ว่า 'ไปละ' มาได้เลย
User: ไปละ
น้องมีเงิน: บะบายจ้า
